# Bookmarks

In [9]:
#!/usr/bin/env python3
"""
transform_json.py

Reads a JSON file, increments every integer value by 1, converts keys
that are ALL-UPPERCASE to sentence-case, and preserves any leading
digit/dot prefix in such keys.

Usage
-----
python transform_json.py input.json output.json
"""
from __future__ import annotations
import json
import re
import sys
from pathlib import Path


# ---------- key-transformation helper ---------------------------------
_UPPER_RE = re.compile(r'^[A-ZԱ-Ֆ]+[\sA-ZԱ-Ֆ]*$')  # Latin & Armenian caps


def fix_key(key: str) -> str:
    """
    If `key` (ignoring leading digit/dot prefix) is ALL-UPPERCASE,
    return the same key but with the text after the prefix sentence-cased.

    Examples
    --------
    "AGE"                       -> "Age"
    "5.4. ԻՆՏԵԳՐԱԼԻ ՀԱՏԿՈՒԹՅՈՒՆՆԵՐԸ"
                                -> "5.4. Ինտեգրալի հատկությունները"
    Anything else is returned unchanged.
    """
    if not isinstance(key, str):
        return key

    # split off an optional numeric/dotted prefix (e.g. "5.4. ")
    m = re.match(r'^([\d.]+\s*)(.+)$', key)
    if m:
        prefix, rest = m.groups()
    else:
        prefix, rest = '', key

    letters_only = ''.join(ch for ch in rest if ch.isalpha())
    if letters_only and _UPPER_RE.fullmatch(letters_only):
        rest = rest.capitalize()

    return prefix + rest


# ---------- recursive JSON transform ----------------------------------
def transform(obj):
    """Recursively apply value & key rules to any JSON-serialisable object."""
    if isinstance(obj, dict):
        return {fix_key(k): transform(v) for k, v in obj.items()}

    if isinstance(obj, list):
        return [transform(item) for item in obj]

    if isinstance(obj, int) and not isinstance(obj, bool):
        return obj + 1

    return obj


# ---------- CLI wrapper -----------------------------------------------
def main(in_path: Path, out_path: Path):
    with in_path.open('r', encoding='utf-8') as f:
        data = json.load(f)

    result = transform(data)

    with out_path.open('w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

    print(f"✅  Transformed JSON written to: {out_path.resolve()}")


if __name__ == "__main__":
    main(Path("bookmarks.json"), Path("fixed_bookmarks.json"))


✅  Transformed JSON written to: C:\Users\hayk_\OneDrive\Desktop\code\manr_munr\musoyan\fixed_bookmarks.json


In [10]:
"""
insert_bookmarks.py
-------------------
Add nested bookmarks to a PDF from a TOC-style JSON file.

Usage
-----
$ python insert_bookmarks.py input.pdf toc.json output.pdf [--offset N]

- input.pdf   : the source PDF
- toc.json    : the JSON file that maps titles → page numbers / subsections
- output.pdf  : destination PDF with bookmarks added
- --offset N  : (optional) integer page offset if PDF and book pages diverge
                e.g. --offset 4  means JSON page 1 → PDF page 5
"""
import argparse
import json
from pathlib import Path

from pypdf import PdfReader, PdfWriter


def add_outline(
    writer: PdfWriter,
    toc: dict[str, "TOCItem"],
    parent=None,
    page_offset: int = 0,
) -> None:
    """
    Recursively walk the TOC dictionary and insert bookmarks.

    A TOCItem is either:
      • int                              → page number
      • {"page": int, "sections": {...}} → nested sections
      • {"page": int, "subsections":{…}} → nested subsections
    """
    for title, item in toc.items():
        # Item is a leaf (plain int) --------------------------
        if isinstance(item, int):
            page_index = item - 1 + page_offset  # JSON is 1-based
            current = writer.add_outline_item(title, page_index, parent=parent)
            continue

        # Item is an object with its own page + nested dict ---
        if isinstance(item, dict) and "page" in item:
            page_index = item["page"] - 1 + page_offset
            current = writer.add_outline_item(title, page_index, parent=parent)

            # recurse into whichever nesting key exists
            for key in ("sections", "subsections"):
                if key in item and isinstance(item[key], dict):
                    add_outline(writer, item[key], parent=current, page_offset=page_offset)
                    break
        else:
            raise ValueError(f"TOC entry for '{title}' has unsupported structure: {item}")


def insert_bookmarks(pdf_in: Path, pdf_out: Path, toc_json: Path, page_offset: int = 0) -> None:
    # ---- read inputs -------------------------------------------------------
    toc = json.loads(Path(toc_json).read_text(encoding="utf-8"))
    reader = PdfReader(str(pdf_in))
    writer = PdfWriter()
    writer.append_pages_from_reader(reader)  # copies all pages

    # ---- build outline -----------------------------------------------------
    add_outline(writer, toc, parent=None, page_offset=page_offset)

    # ---- save --------------------------------------------------------------
    print(pdf_out)
    
    with pdf_out.open("wb") as fh:
        writer.write(fh)
    print(f"✓ Bookmarked PDF written to {pdf_out.resolve()}")


# ---------------------------------------------------------------------------


In [11]:
"""
bookmark_inserter.py
--------------------
Create a new PDF with hierarchical bookmarks taken from a nested-TOC JSON file.

Requirements
------------
pip install pypdf
"""

import json
from pathlib import Path
from typing import Any

from pypdf import PdfReader, PdfWriter


def add_outline(writer: PdfWriter, toc: dict[str, Any], parent=None, *, offset: int = 0) -> None:
    """
    Recursively walk a JSON TOC and create nested bookmarks.

    JSON dialect:
      • Leaf  → "Some title": 17
      • Branch→ "Some section": {"page": 12, "sections" | "subsections": { ... }}
    """
    for title, node in toc.items():
        # --- leaf (plain page number) -------------------------------------
        if isinstance(node, int):
            page = node - 1 + offset
            writer.add_outline_item(title, page, parent=parent)
            continue

        # --- branch (dict with "page" plus children) ----------------------
        if isinstance(node, dict) and "page" in node:
            page = node["page"] - 1 + offset
            this_level = writer.add_outline_item(title, page, parent=parent)

            # recurse into whichever nesting key is present
            for child_key in ("sections", "subsections"):
                if child_key in node and isinstance(node[child_key], dict):
                    add_outline(writer, node[child_key], parent=this_level, offset=offset)
                    break
            else:
                # dict has a page but no children → treat as leaf
                pass
        else:
            raise ValueError(f"Unsupported TOC structure at '{title}': {node}")


def insert_bookmarks(
    pdf_in: str | Path,
    pdf_out: str | Path,
    toc_data: dict[str, Any],
    *,
    page_offset: int = 0,
) -> None:
    """
    Copy pages from pdf_in into pdf_out and attach hierarchical bookmarks.
    """
    reader = PdfReader(str(pdf_in))
    writer = PdfWriter()
    writer.append_pages_from_reader(reader)

    add_outline(writer, toc_data, parent=None, offset=page_offset)

    with Path(pdf_out).open("wb") as fh:
        writer.write(fh)
    print(f"✓ Bookmarked PDF saved to {pdf_out}")


# -------------------------------------------------------------------------
# EDIT THE PATHS BELOW AND RUN THE SCRIPT
# -------------------------------------------------------------------------

if __name__ == "__main__":
    INPUT_PDF   = Path("input.pdf")               # original PDF
    OUTPUT_PDF  = Path("output_bookmarked.pdf")   # destination
    TOC_JSON    = Path("fixed_bookmarks.json")                # the nested TOC we built
    PAGE_OFFSET = 0                               # shift if page-1 ≠ PDF-page-0

    toc = json.loads(TOC_JSON.read_text(encoding="utf-8"))
    insert_bookmarks(INPUT_PDF, OUTPUT_PDF, toc, page_offset=PAGE_OFFSET)


✓ Bookmarked PDF saved to output_bookmarked.pdf


# Footer

In [ ]:
import io
from pypdf import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
from reportlab.lib.utils import ImageReader
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# 1. Register the Armenian font
pdfmetrics.registerFont(
    TTFont('NotoSansArmenian', 'NotoSansArmenian-Black.ttf')
)

def add_footer_with_arm_text_and_two_links(
    input_path: str,
    output_path: str,
    prefix: str,
    icon_path: str,
    links: list[tuple[str, str]],
    connector: str = " և ",  # Armenian “and”
    font_name: str = "NotoSansArmenian",
    font_size: int = 10,
    margin_bottom_mm: float = 12,
    icon_text_gap_pt: float = 5
):
    """
    prefix: leading Armenian text before the icon (e.g. "Խնդրում ենք դիտել ")
    arm_text: Armenian display text after icon (if needed)
    icon_path: path to your 'camera.png'
    links: exactly two (display_text, url) tuples
    connector: text between the two links (default Armenian “ և ”)
    """
    if len(links) != 2:
        raise ValueError("Must provide exactly two links")

    icon = ImageReader(icon_path)
    reader = PdfReader(input_path)
    writer = PdfWriter()

    for page in reader.pages:
        w = float(page.mediabox.width)
        h = float(page.mediabox.height)

        packet = io.BytesIO()
        can = canvas.Canvas(packet, pagesize=(w, h))
        can.setFont(font_name, font_size)

        # Measure widths for centering
        prefix_w    = can.stringWidth(prefix, font_name, font_size)
        icon_size   = font_size
        first_text, first_url   = links[0]
        second_text, second_url = links[1]
        first_w     = can.stringWidth(first_text, font_name, font_size)
        connector_w = can.stringWidth(connector, font_name, font_size)
        second_w    = can.stringWidth(second_text, font_name, font_size)

        total_w = prefix_w + icon_size + icon_text_gap_pt + first_w + connector_w + second_w
        x = (w - total_w) / 2
        y = margin_bottom_mm * mm

        # Draw Armenian prefix
        can.drawString(x, y, prefix)
        x += prefix_w

        # Draw camera icon
        can.drawImage(icon,
                      x,
                      y - (font_size * 0.1),
                      width=icon_size,
                      height=icon_size,
                      mask='auto')
        x += icon_size + icon_text_gap_pt

        # Draw & link first Armenian/Latin text
        can.drawString(x, y, first_text)
        can.linkURL(first_url, (x, y, x + first_w, y + font_size), relative=0)
        x += first_w

        # Draw Armenian connector
        can.drawString(x, y, connector)
        x += connector_w

        # Draw & link second text
        can.drawString(x, y, second_text)
        can.linkURL(second_url, (x, y, x + second_w, y + font_size), relative=0)

        can.save()
        packet.seek(0)

        overlay = PdfReader(packet)
        page.merge_page(overlay.pages[0])
        writer.add_page(page)

    with open(output_path, "wb") as f_out:
        writer.write(f_out)


if __name__ == "__main__":
    add_footer_with_arm_text_and_two_links(
        input_path="input.pdf",
        output_path="output_with_arm_footer.pdf",
        prefix="Կարանք լինքեր դնենք՝",
        icon_path="camera.jpg",
        links=[
            ("էս մեկ", "https://www.youtube.com/watch?v=y4qtOVq_e0U"),
            ("էս էլ երկու", "https://www.youtube.com/watch?v=6_2ZJ4QW_O4")
        ]
    )


https://chatgpt.com/share/e/6851f7fa-1554-8012-89ab-38852aee2d45

bookmarks - https://chatgpt.com/share/e/6852e1ff-a6b0-8012-a32e-1b18fc5a1a01

In [3]:
!pip install pypdf reportlab


   ---------------------------------------- 304.2/304.2 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 2.0/2.0 MB 1.5 MB/s eta 0:00:00


# youtube dfs

In [15]:
import pandas as pd


df = pd.read_csv("res/PLz3NrXxHz_CB7AYnBOd-xSUmoCJ3197oM.csv")

df["Title"].to_clipboard()

In [ ]:
	Title
0	Դաս 9 | Ֆունկցիոնալ շարքի գումարի հատկությունները | Մաթեմատիկական անալիզ 3
1	Դաս 10 | Ֆունկցիոնալ շարքի ինտեգրում և ածանցում | Մաթեմատիկական անալիզ 3
2	Դաս 11 | Աստիճանային շարքեր։ Աբելի թեորեմները | Մաթեմատիկական անալիզ 3
3	Դաս 12 | Անվերջ սահմաններով անիսկական ինտեգրալ | Մաթեմատիկական անալիզ 3
4	Դաս 13 | Անիսկական ինտեգրալի զուգամիտությունը | Մաթեմատիկական անալիզ 3
5	Դաս 14 | Անսահմանափակ ֆունկցիայի անիսկական ինտեգրալ | Մաթեմատիկական անալիզ 3
6	Դաս 15 | Եռանկյունաչափական շարքեր։ Ֆուրիեի շարք | Մաթեմատիկական անալիզ 3
7	Դաս 16 | Ֆուրիեի շարքի զուգամիտության հայտանիշներ | Մաթեմատիկական անալիզ 3
8	Դաս 17 | Ոչ պարբերական ֆունկցիայի Ֆուրիեի շարք | Մաթեմատիկական անալիզ 3
9	Դաս 18 | Ֆեյերի թեորեմը | Մաթեմատիկական անալիզ 3
10	Դաս 19 | Վայերշտրասի թեորեմները։ Զուգահեռանիստի տրոհումներ | Մաթեմատիկական անալիզ 3
11	Դաս 20 | Զուգահեռանիստով տարածված ինտեգրալ | Մաթեմատիկական անալիզ 3
12	Դաս 21 | Ֆունկցիայի տատանում։ Զրո չափի բազմություններ | Մաթեմատիկական անալիզ 3
13	Դաս 22 | Լեբեգի թեորեմը։ Բազմակի ինտեգրալի հատկություններ | Մաթեմատիկական անալիզ 3
14	Դաս 23 | J-չափելի բազմությամբ տարածված ինտեգրալ | Մաթեմատիկական անալիզ 3
15	Դաս 24 | Ֆուբինիի թեորեմը։ Կորագիծ սեղանով կրկնակի ինտեգրալ | Մաթեմատիկական անալիզ 3


# Bookmark matching

In [ ]:
with open("fixed_bookmarks.json", encoding="utf-8") as f:
    bookmarks = json.load(f)
    


{'Տիտղոսաթերթ': 3,
 'Նախաբան': 5,
 '1. Իրական թվեր': {'page': 6,
  'sections': {'1.1. Իրական թվի սահմանումը: իրական թվերի բազմության կարգավորումը': {'page': 7,
    'subsections': {'1.1.1. Իրական թվի սահմանումը': 7,
     '1.1.2. Իրական թվերի բազմության կարգավորումը': 10,
     '1.1.3. Իրական թվերի համակարգի լրիվությունը': 11,
     '1.1.4. ճշգրիտ եզրերի գոյությունը': 12}},
   '1.2. Թվաբանական գործողություններ իրական թվերի հետ': {'page': 14,
    'subsections': {'1.2.1. Իրական թվերի գումարը': 14,
     '1.2.2. Իրական թվերի արտադրյալ': 17}},
   '1.3. Իրական թվերի հետագա հատկությունները': {'page': 19,
    'subsections': {'1.3.1. Արմատի գոյությունը': 19,
     '1.3.2. Իրական ցուցիչով աստիճան': 21,
     '1.3.3. Լոգարի թվի գոյությունը': 23,
     '1.3.4. Իրական թվի տասնորդական ներկայացումը': 24,
     '1.3.5. Թվային առանցք': 27}}}},
 '2. Սահմանների տեսություն': {'page': 30,
  'sections': {'2.1. Հաջորդականության սահման': {'page': 30,
    'subsections': {'2.1.1. Հաջորդականության սահմանի սահմանումը': 3

In [22]:
def extract_subsections(bookmarks):
    subsections = []

    def recurse_sections(sections):
        for key, value in sections.items():
            if isinstance(value, dict) and "page" in value:
                subsections.append([key, value["page"]])
                if "sections" in value:
                    recurse_sections(value["sections"])
                if "subsections" in value:
                    recurse_sections(value["subsections"])

    recurse_sections(bookmarks)
    return subsections

subsections_with_pages = extract_subsections(bookmarks)
print(subsections_with_pages)


[['1. Իրական թվեր', 6], ['1.1. Իրական թվի սահմանումը: իրական թվերի բազմության կարգավորումը', 7], ['1.2. Թվաբանական գործողություններ իրական թվերի հետ', 14], ['1.3. Իրական թվերի հետագա հատկությունները', 19], ['2. Սահմանների տեսություն', 30], ['2.1. Հաջորդականության սահման', 30], ['2.2. Մոնոտոն հաջորդականություններ', 41], ['2.3. Ենթահաջորդականություն եվ մասնակի սահման', 49], ['2.4. Կոշիի զուգամիտության սկզբունքը', 54], ['2.5. Ֆունկցիայի սահման', 55], ['3. Անընհատ ֆունկցիաներ', 68], ['3.1. Ֆունկցիայի անընհատությունը եվ խզուսները', 68], ['3.2. Անընհատ ֆունկցիաների հիմնական հատկությունները', 76], ['3.3. Բորելի լեմման եվ նրա կիրառությունները', 86], ['4. Դիֆերենցիալ հաշիվ', 94], ['4.1. Ածանցյալ', 94], ['4.2. Դիֆերենցիալ', 103], ['4.3. Բարձր կարգի ածանցյալներ եվ դիֆերենցիալներ', 105], ['4.4. Դիֆերենցիալ հաշվի հիսնական թեորեմները', 109], ['4.5. Անորոշությունների բացման լոպիտալի կանոնը', 113], ['4.6. Թեյլորի բանաձեվը', 117], ['4.7. Ֆունկցիաների հետազոտումն ածանցյալների միջոցով', 123], ['4.8. Ուռո

In [26]:
df_bookmarks = pd.DataFrame(subsections_with_pages, columns=["Title", "Page"])

In [27]:
df_bookmarks

,Title,Page
0,1. Իրական թվեր,6
1,1.1. Իրական թվի սահմանումը: իրական թվերի բազմո...,7
2,1.2. Թվաբանական գործողություններ իրական թվերի հետ,14
3,1.3. Իրական թվերի հետագա հատկությունները,19
4,2. Սահմանների տեսություն,30
5,2.1. Հաջորդականության սահման,30
6,2.2. Մոնոտոն հաջորդականություններ,41
7,2.3. Ենթահաջորդականություն եվ մասնակի սահման,49
8,2.4. Կոշիի զուգամիտության սկզբունքը,54
9,2.5. Ֆունկցիայի սահման,55


In [29]:
df_all = pd.read_csv("res/all_playlists.csv")

In [30]:
df_all

,Playlist name,Title,URL,Duration (minutes),Views,Number of likes,Number of dislikes,Number of comments
0,Մաթեմատիկական անալիզ 3,Դաս 9 | Ֆունկցիոնալ շարքի գումարի հատկությունն...,https://www.youtube.com/watch?v=uEhu9ETABhk,51,1641,25,NaN,NaN
1,Մաթեմատիկական անալիզ 3,Դաս 10 | Ֆունկցիոնալ շարքի ինտեգրում և ածանցու...,https://www.youtube.com/watch?v=CIXFmB3sJmA,68,697,12,NaN,3.0
2,Մաթեմատիկական անալիզ 3,Դաս 11 | Աստիճանային շարքեր։ Աբելի թեորեմները ...,https://www.youtube.com/watch?v=PyL9O4pqIV8,58,333,3,NaN,NaN
3,Մաթեմատիկական անալիզ 3,Դաս 12 | Անվերջ սահմաններով անիսկական ինտեգրալ...,https://www.youtube.com/watch?v=lDxky2cvIz4,68,615,16,NaN,NaN
4,Մաթեմատիկական անալիզ 3,Դաս 13 | Անիսկական ինտեգրալի զուգամիտությունը ...,https://www.youtube.com/watch?v=2OAWeagFM1M,58,540,7,NaN,NaN
...,...,...,...,...,...,...,...,...
78,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 16 | Էյլերի բետա և գամմա ինտեգրալներ | Մաթ...,https://www.youtube.com/watch?v=sR8ZU95WiNs,70,175,2,NaN,NaN
79,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 17 | Եռանկյունաչափական շարքեր։ Ֆուրիեի շար...,https://www.youtube.com/watch?v=i7Dqfb_fGNo,77,188,4,NaN,NaN
80,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 18 | Դիրիխլեի ինտեգրալ։ Ռիմանի լեմմա | Մաթ...,https://www.youtube.com/watch?v=9Uk4SYrPgOA,75,207,4,NaN,NaN
81,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 19 | Ֆուրիեի շարքի զուգամիտության հայտանիշ...,https://www.youtube.com/watch?v=uhb-j7HCnzc,70,221,5,NaN,NaN


In [40]:
# !pip install rapidfuzz

                                   Playlist name  \
0                         Մաթեմատիկական անալիզ 3   
1                         Մաթեմատիկական անալիզ 3   
2                         Մաթեմատիկական անալիզ 3   
3                         Մաթեմատիկական անալիզ 3   
4                         Մաթեմատիկական անալիզ 3   
..                                           ...   
78  Մաթեմատիկական անալիզ 4 (տվյալագետների համար)   
79  Մաթեմատիկական անալիզ 4 (տվյալագետների համար)   
80  Մաթեմատիկական անալիզ 4 (տվյալագետների համար)   
81  Մաթեմատիկական անալիզ 4 (տվյալագետների համար)   
82  Մաթեմատիկական անալիզ 4 (տվյալագետների համար)   

                                                Title  \
0   Դաս 9 | Ֆունկցիոնալ շարքի գումարի հատկությունն...   
1   Դաս 10 | Ֆունկցիոնալ շարքի ինտեգրում և ածանցու...   
2   Դաս 11 | Աստիճանային շարքեր։ Աբելի թեորեմները ...   
3   Դաս 12 | Անվերջ սահմաններով անիսկական ինտեգրալ...   
4   Դաս 13 | Անիսկական ինտեգրալի զուգամիտությունը ...   
..                               

In [42]:
df

,Playlist name,Title,URL,Duration (minutes),Views,Number of likes,Number of dislikes,Number of comments,closest_bookmark,bookmark_similarity
0,Մաթեմատիկական անալիզ 3,Դաս 9 | Ֆունկցիոնալ շարքի գումարի հատկությունն...,https://www.youtube.com/watch?v=uEhu9ETABhk,51,1641,25,NaN,NaN,1.3. Իրական թվերի հետագա հատկությունները,85.500000
1,Մաթեմատիկական անալիզ 3,Դաս 10 | Ֆունկցիոնալ շարքի ինտեգրում և ածանցու...,https://www.youtube.com/watch?v=CIXFmB3sJmA,68,697,12,NaN,3.0,5.8. Փոփոխականի փոխարինում եվ սախերով ինտեգրում,85.500000
2,Մաթեմատիկական անալիզ 3,Դաս 11 | Աստիճանային շարքեր։ Աբելի թեորեմները ...,https://www.youtube.com/watch?v=PyL9O4pqIV8,58,333,3,NaN,NaN,4.4. Դիֆերենցիալ հաշվի հիսնական թեորեմները,85.500000
3,Մաթեմատիկական անալիզ 3,Դաս 12 | Անվերջ սահմաններով անիսկական ինտեգրալ...,https://www.youtube.com/watch?v=lDxky2cvIz4,68,615,16,NaN,NaN,5.1. Անորոշ ինտեգրալ,85.500000
4,Մաթեմատիկական անալիզ 3,Դաս 13 | Անիսկական ինտեգրալի զուգամիտությունը ...,https://www.youtube.com/watch?v=2OAWeagFM1M,58,540,7,NaN,NaN,9.1. Թվային շարքի գումարը եվ զուգամիտությունը,85.500000
...,...,...,...,...,...,...,...,...,...,...
78,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 16 | Էյլերի բետա և գամմա ինտեգրալներ | Մաթ...,https://www.youtube.com/watch?v=sR8ZU95WiNs,70,175,2,NaN,NaN,5. Ինտեգրալ հաշիվ,52.941176
79,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 17 | Եռանկյունաչափական շարքեր։ Ֆուրիեի շար...,https://www.youtube.com/watch?v=i7Dqfb_fGNo,77,188,4,NaN,NaN,9.5. Թեյլորի շարք,85.500000
80,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 18 | Դիրիխլեի ինտեգրալ։ Ռիմանի լեմմա | Մաթ...,https://www.youtube.com/watch?v=9Uk4SYrPgOA,75,207,4,NaN,NaN,5. Ինտեգրալ հաշիվ,58.235294
81,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 19 | Ֆուրիեի շարքի զուգամիտության հայտանիշ...,https://www.youtube.com/watch?v=uhb-j7HCnzc,70,221,5,NaN,NaN,2.4. Կոշիի զուգամիտության սկզբունքը,85.500000


## Fuzzy

In [43]:
import pandas as pd
from rapidfuzz import process, fuzz   # pip install rapidfuzz

def add_fuzzy_matches(
    df: pd.DataFrame,
    feature_col: str,
    reference_list: list[str],
    scorers: dict[str, callable] | None = None,
    case_insensitive: bool = True,
) -> pd.DataFrame:
    """
    Append fuzzy-matching results for *several* scorers at once.

    For each (name, scorer) pair in `scorers`, two columns are added:

        •  <name>_match  – closest string from `reference_list`
        •  <name>_score  – similarity score (0-100)

    Parameters
    ----------
    df : pd.DataFrame
    feature_col : str
        Column to match.
    reference_list : list[str]
        Candidate strings.
    scorers : dict[str, callable], optional
        Keys are short names that become the column prefixes;
        values are RapidFuzz scorer callables.
        Defaults to four commonly used strategies.
    case_insensitive : bool, optional
        If True (default), comparisons ignore case.
    """
    # --- default strategies --------------------------------------------------
    if scorers is None:
        scorers = {
            "wratio": fuzz.WRatio,             # weighted (good all-rounder)
            "qratio": fuzz.QRatio,             # quick ratio (fast)
            "tsort": fuzz.token_sort_ratio,    # ignore word order
            "tset":  fuzz.token_set_ratio,     # ignore word order + dupes
        }

    # Optionally lower-case everything once up front
    ref = [s.lower() for s in reference_list] if case_insensitive else reference_list

    def _best_match(val: str, scorer):
        """Return (match, score) against `ref` using chosen scorer."""
        if pd.isna(val) or val == "":
            return None, 0
        query = val.lower() if case_insensitive else val
        match, score, _ = process.extractOne(query, ref, scorer=scorer)
        return match, score

    # ------------------------------------------------------------------------
    out = df.copy()
    for label, scorer in scorers.items():
        match_col = f"{label}_match"
        score_col = f"{label}_score"
        out[[match_col, score_col]] = (
            out[feature_col]
            .apply(_best_match, scorer=scorer)
            .apply(pd.Series)
        )

    return out


In [46]:
df = add_fuzzy_matches(df_all, "Title", df_bookmarks["Title"].tolist())
                    #  scorer=fuzz.token_sort_ratio,
                    #  match_col="closest_bookmark",
                    #  score_col="bookmark_similarity")

print(df)

                                   Playlist name  \
0                         Մաթեմատիկական անալիզ 3   
1                         Մաթեմատիկական անալիզ 3   
2                         Մաթեմատիկական անալիզ 3   
3                         Մաթեմատիկական անալիզ 3   
4                         Մաթեմատիկական անալիզ 3   
..                                           ...   
78  Մաթեմատիկական անալիզ 4 (տվյալագետների համար)   
79  Մաթեմատիկական անալիզ 4 (տվյալագետների համար)   
80  Մաթեմատիկական անալիզ 4 (տվյալագետների համար)   
81  Մաթեմատիկական անալիզ 4 (տվյալագետների համար)   
82  Մաթեմատիկական անալիզ 4 (տվյալագետների համար)   

                                                Title  \
0   Դաս 9 | Ֆունկցիոնալ շարքի գումարի հատկությունն...   
1   Դաս 10 | Ֆունկցիոնալ շարքի ինտեգրում և ածանցու...   
2   Դաս 11 | Աստիճանային շարքեր։ Աբելի թեորեմները ...   
3   Դաս 12 | Անվերջ սահմաններով անիսկական ինտեգրալ...   
4   Դաս 13 | Անիսկական ինտեգրալի զուգամիտությունը ...   
..                               

In [50]:
cols_drop = ["URL", "Views", "Duration (minutes)", "Number of likes", "Number of dislikes", "Number of comments"]


In [51]:
df.drop(columns=cols_drop)

,Playlist name,Title,closest_bookmark,bookmark_similarity,wratio_match,wratio_score,qratio_match,qratio_score,tsort_match,tsort_score,tset_match,tset_score
0,Մաթեմատիկական անալիզ 3,Դաս 9 | Ֆունկցիոնալ շարքի գումարի հատկությունն...,1.3. Իրական թվերի հետագա հատկությունները,85.500000,1.3. իրական թվերի հետագա հատկությունները,85.500000,3.2. անընհատ ֆունկցիաների հիմնական հատկություն...,50.000000,3.2. անընհատ ֆունկցիաների հիմնական հատկություն...,58.064516,5.4. ինտեգրալի հատկությունները,66.666667
1,Մաթեմատիկական անալիզ 3,Դաս 10 | Ֆունկցիոնալ շարքի ինտեգրում և ածանցու...,5.8. Փոփոխականի փոխարինում եվ սախերով ինտեգրում,85.500000,5.8. փոփոխականի փոխարինում եվ սախերով ինտեգրում,85.500000,4.7. ֆունկցիաների հետազոտումն ածանցյալների միջ...,44.262295,3.2. անընհատ ֆունկցիաների հիմնական հատկություն...,44.262295,3.2. անընհատ ֆունկցիաների հիմնական հատկություն...,45.000000
2,Մաթեմատիկական անալիզ 3,Դաս 11 | Աստիճանային շարքեր։ Աբելի թեորեմները ...,4.4. Դիֆերենցիալ հաշվի հիսնական թեորեմները,85.500000,4.4. դիֆերենցիալ հաշվի հիսնական թեորեմները,85.500000,9.1. թվային շարքի գումարը եվ զուգամիտությունը,40.000000,9.4. զուգամետ շարքերի հիսնական հատկությունները,43.103448,4.4. դիֆերենցիալ հաշվի հիսնական թեորեմները,45.454545
3,Մաթեմատիկական անալիզ 3,Դաս 12 | Անվերջ սահմաններով անիսկական ինտեգրալ...,5.1. Անորոշ ինտեգրալ,85.500000,5. ինտեգրալ հաշիվ,85.500000,1.1. իրական թվի սահմանումը: իրական թվերի բազմո...,44.444444,5.2. որոշյալ ինտեգրալի սահմանումը եվ գոյության...,43.750000,5. ինտեգրալ հաշիվ,64.000000
4,Մաթեմատիկական անալիզ 3,Դաս 13 | Անիսկական ինտեգրալի զուգամիտությունը ...,9.1. Թվային շարքի գումարը եվ զուգամիտությունը,85.500000,5.4. ինտեգրալի հատկությունները,85.500000,1.2. թվաբանական գործողություններ իրական թվերի հետ,47.058824,1.3. իրական թվերի հետագա հատկությունները,50.909091,9.1. թվային շարքի գումարը եվ զուգամիտությունը,52.459016
...,...,...,...,...,...,...,...,...,...,...,...,...
78,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 16 | Էյլերի բետա և գամմա ինտեգրալներ | Մաթ...,5. Ինտեգրալ հաշիվ,52.941176,5. ինտեգրալ հաշիվ,58.235294,5.6. ինտեգրալ հաշվի հիսնական բանաձեվը (նյուտոն...,39.370079,5.6. ինտեգրալ հաշվի հիսնական բանաձեվը (նյուտոն...,40.944882,5.6. ինտեգրալ հաշվի հիսնական բանաձեվը (նյուտոն...,41.600000
79,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 17 | Եռանկյունաչափական շարքեր։ Ֆուրիեի շար...,9.5. Թեյլորի շարք,85.500000,9.5. թեյլորի շարք,85.500000,1.2. թվաբանական գործողություններ իրական թվերի հետ,37.500000,7.4. անընդհատ ֆունկցիաների հիսնական հատկությու...,43.076923,7.4. անընդհատ ֆունկցիաների հիսնական հատկությու...,43.750000
80,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 18 | Դիրիխլեի ինտեգրալ։ Ռիմանի լեմմա | Մաթ...,5. Ինտեգրալ հաշիվ,58.235294,5. ինտեգրալ հաշիվ,63.529412,5.6. ինտեգրալ հաշվի հիսնական բանաձեվը (նյուտոն...,40.944882,5.6. ինտեգրալ հաշվի հիսնական բանաձեվը (նյուտոն...,40.944882,5.6. ինտեգրալ հաշվի հիսնական բանաձեվը (նյուտոն...,41.600000
81,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 19 | Ֆուրիեի շարքի զուգամիտության հայտանիշ...,2.4. Կոշիի զուգամիտության սկզբունքը,85.500000,2.4. կոշիի զուգամիտության սկզբունքը,85.500000,"9.7. կումերի, բերտրանի եվ գաուսի հայտանիշները",41.269841,9.4. զուգամետ շարքերի հիսնական հատկությունները,48.818898,2.4. կոշիի զուգամիտության սկզբունքը,57.142857


In [ ]:
Number of likes

Number of dislikes

Number of comments


,Playlist name,Title,URL,Duration (minutes),Views,Number of likes,Number of dislikes,Number of comments,closest_bookmark,bookmark_similarity
0,Մաթեմատիկական անալիզ 3,Դաս 9 | Ֆունկցիոնալ շարքի գումարի հատկությունն...,https://www.youtube.com/watch?v=uEhu9ETABhk,51,1641,25,NaN,NaN,1.3. Իրական թվերի հետագա հատկությունները,85.500000
1,Մաթեմատիկական անալիզ 3,Դաս 10 | Ֆունկցիոնալ շարքի ինտեգրում և ածանցու...,https://www.youtube.com/watch?v=CIXFmB3sJmA,68,697,12,NaN,3.0,5.8. Փոփոխականի փոխարինում եվ սախերով ինտեգրում,85.500000
2,Մաթեմատիկական անալիզ 3,Դաս 11 | Աստիճանային շարքեր։ Աբելի թեորեմները ...,https://www.youtube.com/watch?v=PyL9O4pqIV8,58,333,3,NaN,NaN,4.4. Դիֆերենցիալ հաշվի հիսնական թեորեմները,85.500000
3,Մաթեմատիկական անալիզ 3,Դաս 12 | Անվերջ սահմաններով անիսկական ինտեգրալ...,https://www.youtube.com/watch?v=lDxky2cvIz4,68,615,16,NaN,NaN,5.1. Անորոշ ինտեգրալ,85.500000
4,Մաթեմատիկական անալիզ 3,Դաս 13 | Անիսկական ինտեգրալի զուգամիտությունը ...,https://www.youtube.com/watch?v=2OAWeagFM1M,58,540,7,NaN,NaN,9.1. Թվային շարքի գումարը եվ զուգամիտությունը,85.500000
...,...,...,...,...,...,...,...,...,...,...
78,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 16 | Էյլերի բետա և գամմա ինտեգրալներ | Մաթ...,https://www.youtube.com/watch?v=sR8ZU95WiNs,70,175,2,NaN,NaN,5. Ինտեգրալ հաշիվ,52.941176
79,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 17 | Եռանկյունաչափական շարքեր։ Ֆուրիեի շար...,https://www.youtube.com/watch?v=i7Dqfb_fGNo,77,188,4,NaN,NaN,9.5. Թեյլորի շարք,85.500000
80,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 18 | Դիրիխլեի ինտեգրալ։ Ռիմանի լեմմա | Մաթ...,https://www.youtube.com/watch?v=9Uk4SYrPgOA,75,207,4,NaN,NaN,5. Ինտեգրալ հաշիվ,58.235294
81,Մաթեմատիկական անալիզ 4 (տվյալագետների համար),Դաս 19 | Ֆուրիեի շարքի զուգամիտության հայտանիշ...,https://www.youtube.com/watch?v=uhb-j7HCnzc,70,221,5,NaN,NaN,2.4. Կոշիի զուգամիտության սկզբունքը,85.500000
